In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
mnist = input_data.read_data_sets('./data/MNIST_data/', one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
X = tf.placeholder('float', [None, 784])  # 784 is the image size
Y = tf.placeholder('float', [None, 10])  # 10 labels

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# define weight parameter
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))
# initializing weight parameters
w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

In [4]:
# define model
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # the fist full connect layer
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))
    # the second full connect layer
    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    return tf.matmul(h2, w_o)

In [5]:
# generate net and get the predictions
p_keep_input = tf.placeholder('float')
p_keep_hidden = tf.placeholder('float')
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

# define cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [6]:
# train and save model
ckpt_dir = './data/mnist_ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

# define a counter to count the training iterations, set its trainable=False, forbidden to train the counter
global_step = tf.Variable(0, name='global_step', trainable=False)
# Call this after declaring all tf.Variables, Variables after saver would not be saved
saver = tf.train.Saver()

In [8]:
# training
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    start = global_step.eval() # obtain the initialized value of global_step
    print('Start from:', start)
    
    for i in range(start, 100):
        # batch_size = 128
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
            
            global_step.assign(i).eval() # update counter
        saver.save(sess, ckpt_dir + '/model.ckpt', global_step=global_step)  # save model

('Start from:', 0)


KeyboardInterrupt: 

In [9]:
# restore model
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path) # load all the parameters
        # now you can continue train the model or do prediction

./data/mnist_ckpt_dir/model.ckpt-63
INFO:tensorflow:Restoring parameters from ./data/mnist_ckpt_dir/model.ckpt-63


In [10]:
# save graph
with tf.Session() as sess:
    tf.train.write_graph(sess.graph_def, './data/mnist_ckpt_dir/tfmodel', 'train.pbtxt')

In [ ]:
# read graph
'''
with tf.Session() as _sess:
    with tf.gfile.FastGFile('/tmp/tfmodel/train.pbtxt', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _sess.graph.as_default()
        tf.import_graph_def(graph_def, name='tfgraph')
'''